Job Application Assistant

Overview

This tool helps with job applications by comparing a resume to a job description. It then creates a personalized cover letter, points out any skill gaps, and generates practice interview questions.

Input: Resume (PDF/as location ) + Job description (as text)

Output: PDF report with tailored cover letter, skill gap analysis, interview preparation questions

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_openai langchain_community langchain_core tavily-pytJob Application Assistant

Overview

This tool helps with job applications by comparing a resume to a job description. It then creates a personalized cover letter, points out any skill gaps, and generates practice interview questions.

Input: Resume (PDF/as location ) + Job description (as text)

Output: PDF report with tailored cover letter, skill gap analysis, interview preparation questionshon pydantic langchain-tavily pypdf fpdf

Setup

In [ ]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o", temperature=0)

/home/jai/study_material/sem7/llm/temp/.venv/lib64/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [ ]:

_set_env("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "job-application-assistant"

In [ ]:

_set_env("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "job-application-assistant"

PDF Resume Reader

Upload and extract text from PDF resume files.

In [ ]:
from pypdf import PdfReader

def read_resume_pdf(pdf_path: str) -> str:
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text.strip()

Data Models (Structured Output)

We define Pydantic models to ensure structured, validated output from the LLM.

In [ ]:
from typing import List, Optional
from typing_extensions import TypedDict
from pydantic import BaseModel, Field

class Experience(BaseModel):
    company: str = Field(description="Company name")
    role: str = Field(description="Job title/role")
    duration: str = Field(description="Duration of employment")
    highlights: List[str] = Field(description="Key achievements and responsibilities")

class Education(BaseModel):
    institution: str = Field(description="School/University name")
    degree: str = Field(description="Degree obtained")
    year: str = Field(description="Year of graduation")
    gpa: Optional[str] = Field(default=None, description="GPA if mentioned")

class ParsedResume(BaseModel):
    name: str = Field(description="Candidate's full name")
    email: Optional[str] = Field(default=None, description="Email address")
    phone: Optional[str] = Field(default=None, description="Phone number")
    summary: str = Field(description="Professional summary")
    skills: List[str] = Field(description="Technical and soft skills")
    experience: List[Experience] = Field(description="Work experience")
    education: List[Education] = Field(description="Educational background")
    certifications: List[str] = Field(default=[], description="Certifications")

class JobRequirements(BaseModel):
    title: str = Field(description="Job title")
    company: str = Field(description="Company name")
    required_skills: List[str] = Field(description="Required technical skills")
    preferred_skills: List[str] = Field(description="Nice-to-have skills")
    responsibilities: List[str] = Field(description="Key job responsibilities")
    qualifications: List[str] = Field(description="Required qualifications")
    experience_years: Optional[str] = Field(default=None, description="Years of experience required")

class SkillMatch(BaseModel):
    skill: str = Field(description="The skill being analyzed")
    status: str = Field(description="'matched', 'partial', or 'missing'")
    evidence: Optional[str] = Field(default=None, description="Evidence from resume if matched")

class SkillGapAnalysis(BaseModel):
    match_score: int = Field(description="Overall match percentage 0-100")
    matched_skills: List[SkillMatch] = Field(description="Skills that match")
    partial_matches: List[SkillMatch] = Field(description="Transferable/related skills")
    missing_skills: List[SkillMatch] = Field(description="Skills to develop")
    recommendations: List[str] = Field(description="Suggestions to bridge gaps")

class CompanyResearch(BaseModel):
    company_name: str = Field(description="Company name")
    description: str = Field(description="What the company does")
    culture: str = Field(description="Company culture and values")
    recent_news: List[str] = Field(description="Recent news or achievements")
    interview_tips: List[str] = Field(description="Tips for interviewing at this company")

class CoverLetter(BaseModel):
    greeting: str = Field(description="Opening greeting")
    opening_paragraph: str = Field(description="Hook and introduction")
    body_paragraphs: List[str] = Field(description="Main content paragraphs")
    closing_paragraph: str = Field(description="Call to action and closing")
    signature: str = Field(description="Sign-off and name")
    
    def to_text(self) -> str:
        body = "\n\n".join(self.body_paragraphs)
        return f"{self.greeting}\n\n{self.opening_paragraph}\n\n{body}\n\n{self.closing_paragraph}\n\n{self.signature}"

class InterviewQuestion(BaseModel):
    question: str = Field(description="The interview question")
    category: str = Field(description="Category: technical, behavioral, situational, company-specific")
    suggested_answer: str = Field(description="Suggested answer approach based on resume")
    tips: str = Field(description="Tips for answering")

class InterviewPrep(BaseModel):
    technical_questions: List[InterviewQuestion] = Field(description="Technical questions")
    behavioral_questions: List[InterviewQuestion] = Field(description="Behavioral questions")
    company_specific: List[InterviewQuestion] = Field(description="Company-specific questions")
    questions_to_ask: List[str] = Field(description="Questions candidate should ask")

LangGraph State

The state holds all information as it flows through our multi-agent system.

In [ ]:
class ApplicationState(TypedDict):
    resume_text: str
    job_description: str
    parsed_resume: Optional[ParsedResume]
    job_requirements: Optional[JobRequirements]
    company_research: Optional[CompanyResearch]
    skill_gap: Optional[SkillGapAnalysis]
    cover_letter: Optional[CoverLetter]
    interview_prep: Optional[InterviewPrep]
    human_feedback: Optional[str]

Node 1: Parse Resume

Extract structured information from the resume using prompting and structured output.

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

resume_parser_prompt = """You are an expert resume parser. Analyze the provided resume and extract all relevant information.

Be thorough in extracting:
- All skills mentioned (technical, soft skills, tools, frameworks)
- Complete work history with achievements
- Educational background
- Certifications and additional qualifications

If information is not explicitly stated, make reasonable inferences but note them."""

def parse_resume(state: ApplicationState) -> dict:
    structured_llm = llm.with_structured_output(ParsedResume)
    
    result = structured_llm.invoke([
        SystemMessage(content=resume_parser_prompt),
        HumanMessage(content=f"Parse this resume:\n\n{state['resume_text']}")
    ])
    
    return {"parsed_resume": result}